### Purpose:

Using Descriptive Statistics, apply different calculations to measure insights such as:

+ Countries that average the highest/lowest spending
+ Countries above and below average
+ Grouping coutries by percentiles
+ More to add as the project progresses

What concepts will be practiced?

+ Estimates of Location
+ Estimates of Variability
+ Exploring Data Distribution
+ Binary and Categorical Data

In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
from scipy import stats
from scipy.stats import trim_mean
from statsmodels import robust

# reading in the csv
df = pd.read_csv('data/state_csv.csv')
df.head(5)

In [ ]:
# understanding the dataset
df.info()

In [ ]:
# rounding and transposing the basic descriptive analyics
round(df.describe().T,2)

## Calculating Different Estimates of Location

### For the first exercise, will use the year 2016

#### Mean 
+ Sum of all values divided by the number of values

#### Median
+ (50th Percentile) The value where such value of the data is above, and the other half below

### Trimmed Mean
+ The average of all values after dropping a fixed set of extreme values

### Weighted Mean
+ The sum of all values times a weight divided by the sum of the weights

In [ ]:
# renaming the column Murder.Rate to Murder Rate
df = df.rename({"Murder.Rate" : "Murder Rate"}, axis = 1)

# looking at the first five columns to also ensure the renaming also worked
df.head(5)

In [ ]:
# calculating the average
mean_pop = round(df['Population'].mean(), 2)
mean_pop

In [ ]:
# calculating the median
median_pop = round(df['Population'].median(), 2)
median_pop

In [ ]:
# calculating the trimmed mean
trim_mean_pop = round(stats.trim_mean(df['Population'], 0.1),2)
trim_mean_pop

In [ ]:
# calculating the trim mean in a different way to confirm it matches and it works
tr_mean_2 = round(trim_mean(df['Population'], 0.1), 2)
tr_mean_2

In [ ]:
# calculating the weighted mean
weighted_mean = round(np.average(df['Murder Rate'], weights = df['Population']),2)
weighted_mean

In [ ]:
import wquantiles

# calculating the weighted median
weighted_median = round(wquantiles.median(df['Murder Rate'], weights = df['Population']), 2)
weighted_median

## Estimates of Variability 

### Deviations
+ Measure that is used to find the difference betwen the observed value and the expected value of a variable. In other words, it is the distance from the centre point
+ Value - mean of the data

### Variance 
+ The sum of squared deviatons from the mean divided by n - 1 where n is the number of data values
+ Sum of deviations/(count of data values - 1)

### Standard Deviation
+ The Square Root of the Variance

### Mean Absolute Deviation
+ The mean of the absolute values of the deviations from the mean

### Percentile
+ The value such that P percent of the values take on this value or less (100-P) percent to take on this value or more

### Interquartile Range (IQR)
+ The difference between the 75th percentile and the 25th percentile

In [ ]:
# calculating the standard deviation of the population

std_deviation = round(df['Population'].std(),2)
std_deviation

In [ ]:
# calculating the interquartile range of the population

iqr = round(df['Population'].quantile(0.75) - df['Population'].quantile(0.25), 2)
iqr

In [ ]:
# using numpy to find mean absolute deviation

data = df['Population']
mean = np.mean(data)
abs_dev = np.absolute(data - mean)
mad = round(np.mean(abs_dev), 2)
mad

In [ ]:
# now lets use pandas .mad() to see if we get the same answer

p_mad = round(df['Population'].mad(), 2)
p_mad

### Percentiles

In [ ]:
# percentiles of population by state

df['Population'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])

In [ ]:
# creating a boxplot to visualize the distribution of population by state

ax = (df['Population']/1_000_000).plot.box(figsize=(3, 4))
ax.set_ylabel('Population (millions)')

plt.tight_layout()
plt.show();

### Key Takeaways from the Boxplot above

+ The horizontal line is the median which is close to 5 million
+ The top and bottom half of the both are the 75th and the 25th percentile
+ The line, also known as whiskers, at beginning at the start and ending at the top closer to 13 Million represent the range for the bulk of the data (excluding outliers)

### Frequency Table and Histogram

+ Divides up the variable range into equally spaced segments and tells us how many values fall within each segment
+ The function pandas.cut() creates a series that maps the values into the segment
+ Using the method value_counts() we can get the frequency tables

In [ ]:
binned_population = pd.cut(df['Population'],10)
binned_population.value_counts()

In [ ]:
binned_population.name = 'binnedpopulation'
df = pd.concat([df, binned_population], axis = 1)
df = df.sort_values(by = 'Population')

groups = []
for group, subset in df.groupby(by='binnedpopulation'):
    groups.append({
        'BinRange': group,
        'Count': len(subset),
        'States': ','.join(subset.Abbreviation)
    })
print(pd.DataFrame(groups))

### Takeaways

+ The least populous state is Wyoming (Top Left) with 536,626 people 
+ The most populated state is California with 37,253,956 people
+ This gives us a range of 37,253,956 - 536,626 which gives us 36,690,330 to divide into equal size bins

### Next Steps

+ Create a visual for the frequency table above

In [ ]:
ax = (df['Population'] / 1_000_000).plot.hist(figsize = (4,4))
ax.set_xlabel('Population (millions)')

plt.tight_layout()
plt.show();

### Density Estimates

+ Shows the distribution of data values as a continous line
+ A density plot can be thought of as a smoothed histogram
+ Although it is typically computed directly from the data as a kernel density estimate

In [ ]:
ax = df['Murder Rate'].plot.hist(density = True, xlim = [0,12], bins = range(1,12), figsize = (4,4))

df['Murder Rate'].plot.density(ax = ax)
ax.set_xlabel('Murder Rate (per 100,000)')
plt.show();

### Exploring Binary and Categorical Data

+ For categorical data, simple proportions or percentages tell the story of data

### Key Terms for Exploring Categorical Data

+ Mode: Most commonly occurring category or value in a dataset
+ Expected Value: When the categories are associated with a numeric value
+ Bar Charts: The frequency or proportion for each category plotted as bars
+ Pie Charts: The frequency or proportion for each category plotted as wedges in a pie

### Next Steps

+ Import airport delays per year by cause for Dallas/Fort Worth(DFW)
+ Delays are categorized as being due to factors under:

- Carrier Control
- Air Traffic Control
- Control (ATC) System Delays
- Weather Security
- Late Inbound Aircraft

In [ ]:
dfw = pd.read_csv('data/airport_delays.csv')

In [ ]:
dfw.info()

In [ ]:
print(100 * dfw / dfw.values.sum())

### Bar Charts

+ Are a common visualization tool for displaying single categorical variables
+ Categories are listed on the x-axis and frequencies or proportions on the y-axis

In [ ]:
ax = dfw.T.plot.bar(figsize=(4, 4), legend=False)
ax.set_xlabel('Cause of delay')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show();

### Correlation

+ The process of establishing a relationship or connection between two or more measures

#### Key Terms for Correlation

+ Correlation Coefficient: A matric that measures the extext to which numeric variables are associated with one another (ranges from -1 to +1)
+ Correlation Matrix: A table where the variables are shown on both rows and columns, and the cell values are the correlations between the variables

In [ ]:
# reading in sp_500 data csv
sp_data = pd.read_csv('data/sp500_data.csv', index_col = 0)

# reading in sp_sectors csv
sp_sec = pd.read_csv('data/sp500_sectors.csv')

In [ ]:
# understanding the data landscape in sp_data
sp_data.info()

In [ ]:
# understanding the data landscape in sp_sec
sp_sec.info()

In [ ]:
# renaming first column from Unnamed: 0 to Date
sp_data = sp_data.rename({"Unnamed: 0" : "Date"}, axis = 1)

# switching Date to datetime
# sp_data['Date'] = pd.to_datetime(sp_data['Date'])

# looking at the first five rows of the dataset to ensure it worked
sp_data.head(5)

In [ ]:
# looking at the first five records of the sector dataset
sp_sec.head(5)

In [ ]:
symbols = sp_sec[sp_sec['sector'] == 'telecommunications_services']['symbol']
telecom = sp_data.loc[sp_data.index > '2012-07-01', symbols]
telecom.corr()